EDA

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# Display basic information about the dataset
print("Dataset Info:")
print(df.info())
print("\n")

# Display summary statistics
print("Summary Statistics:")
print(df.describe())
print("\n")

# Check for missing values
print("Missing Values:")
print(df.isnull().sum())
print("\n")

# Display the distribution of the target variable
print("Target Variable Distribution:")
print(df['Diabetes_binary'].value_counts(normalize=True))
print("\n")

# Correlation matrix
corr_matrix = df.corr()
print("Top 10 correlations with Diabetes_binary:")
print(corr_matrix['Diabetes_binary'].sort_values(ascending=False).head(10))
print("\n")

# Skewness of numerical features
print("Skewness of numerical features:")
print(df.select_dtypes(include=[np.number]).skew())
print("\n")

# Chi-square test for categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns
print("Chi-square test results for categorical variables:")
for col in categorical_cols:
    chi2, p_value, dof, expected = stats.chi2_contingency(pd.crosstab(df['Diabetes_binary'], df[col]))
    print(f"{col}: chi2={chi2:.2f}, p-value={p_value:.4f}")
print("\n")

# T-test for numerical variables
numerical_cols = df.select_dtypes(include=[np.number]).columns
numerical_cols = numerical_cols.drop('Diabetes_binary')
print("T-test results for numerical variables:")
for col in numerical_cols:
    t_stat, p_value = stats.ttest_ind(df[df['Diabetes_binary']==0][col], 
                                      df[df['Diabetes_binary']==1][col])
    print(f"{col}: t-statistic={t_stat:.2f}, p-value={p_value:.4f}")
print("\n")

# Value counts for categorical variables
print("Value counts for categorical variables:")
for col in categorical_cols:
    print(f"\n{col}:")
    print(df[col].value_counts(normalize=True))

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   ID                    253680 non-null  int64
 1   Diabetes_binary       253680 non-null  int64
 2   HighBP                253680 non-null  int64
 3   HighChol              253680 non-null  int64
 4   CholCheck             253680 non-null  int64
 5   BMI                   253680 non-null  int64
 6   Smoker                253680 non-null  int64
 7   Stroke                253680 non-null  int64
 8   HeartDiseaseorAttack  253680 non-null  int64
 9   PhysActivity          253680 non-null  int64
 10  Fruits                253680 non-null  int64
 11  Veggies               253680 non-null  int64
 12  HvyAlcoholConsump     253680 non-null  int64
 13  AnyHealthcare         253680 non-null  int64
 14  NoDocbcCost           253680 non-null  int64
 15  GenHlth             

Feature Engineering

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Read the data file
df = pd.read_csv("https://raw.githubusercontent.com/NumanESchulich/SchulichDataScience/main/Data%20Science%20I%20(MBAN%206110T)/Group%20Assignment/Datasets/Full%20Dataset%20(RAW).csv")

# 1. Create interaction terms
df['BMI_x_Age'] = df['BMI'] * df['Age']
df['HighBP_x_HighChol'] = df['HighBP'] * df['HighChol']

# 2. Bin BMI
def bmi_category(bmi):
    if bmi < 18.5:
        return 0
    elif bmi < 25:
        return 1
    elif bmi < 30:
        return 2
    else:
        return 3

df['BMI_Category'] = df['BMI'].apply(bmi_category)

# 3. Create composite health score
df['Health_Score'] = df['GenHlth'] + df['HighBP'] + df['HighChol']

# 4. Age groups
def age_group(age):
    if age <= 4:
        return 0
    elif age <= 8:
        return 1
    else:
        return 2

df['Age_Group'] = df['Age'].apply(age_group)

# 5. Lifestyle score
df['Lifestyle_Score'] = df['PhysActivity'] + df['Fruits'] + df['Veggies'] - df['HvyAlcoholConsump']

# 6. Socioeconomic status
df['SES'] = df['Education'] + df['Income']

# 7. Healthcare access score
df['Healthcare_Access'] = df['AnyHealthcare'] - df['NoDocbcCost']

# 8. Overall health score
df['Overall_Health'] = df['MentHlth'] + df['PhysHlth'] + df['GenHlth']

# 9. Log transformations for skewed features
skewed_features = ['BMI', 'MentHlth', 'PhysHlth']
for feature in skewed_features:
    df[f'{feature}_Log'] = np.log1p(df[feature])

# 10. Polynomial features
df['BMI_Squared'] = df['BMI'] ** 2
df['Age_Squared'] = df['Age'] ** 2

# 11. Impact encoding for binary features
binary_features = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

for feature in binary_features:
    impact = df.groupby(feature)['Diabetes_binary'].mean()
    df[f'{feature}_Impact'] = df[feature].map(impact)

# Create lists of numeric and categorical columns
numeric_features = ['BMI', 'MentHlth', 'PhysHlth', 'Age', 'BMI_x_Age', 'Health_Score', 'Lifestyle_Score', 'SES', 'Healthcare_Access', 'Overall_Health', 'BMI_Log', 'MentHlth_Log', 'PhysHlth_Log', 'BMI_Squared', 'Age_Squared'] + [f'{feature}_Impact' for feature in binary_features]
categorical_features = ['BMI_Category', 'Age_Group']

# Create preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Prepare the features and target
X = df.drop(['Diabetes_binary', 'ID'], axis=1)
y = df['Diabetes_binary']

# Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

# Convert to dataframe for easier handling
feature_names = numeric_features + categorical_features
X_transformed_df = pd.DataFrame(X_transformed, columns=feature_names)

print("Transformed dataset shape:", X_transformed_df.shape)
print("\nFirst few rows of the transformed dataset:")
print(X_transformed_df.head())

Transformed dataset shape: (253680, 31)

First few rows of the transformed dataset:
        BMI  MentHlth  PhysHlth       Age  BMI_x_Age  Health_Score  \
0  1.757936  1.998592  1.233999  0.316900   1.353202      2.389548   
1 -0.511806 -0.429630 -0.486592 -0.337933  -0.532352     -0.239590   
2 -0.057858  3.617407  2.954590  0.316900   0.252446      2.389548   
3 -0.209174 -0.429630 -0.486592  0.971733   0.711094     -0.239590   
4 -0.663122 -0.024926 -0.486592  0.971733   0.374752      0.417695   

   Lifestyle_Score       SES  Healthcare_Access  Overall_Health  ...  \
0        -1.257865 -1.540836           0.342017        2.023357  ...   
1        -1.257865 -1.540836          -4.796284       -0.500273  ...   
2        -1.257865  0.336260          -2.227134        3.970158  ...   
3         0.937319 -0.789998           0.342017       -0.572377  ...   
4         0.937319 -0.789998           0.342017       -0.356066  ...   

   PhysActivity_Impact  Fruits_Impact  Veggies_Impact  \
0    

XGBoost + LightGBM Models

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE

# Assume X_selected and y are already defined from previous steps

# Reduce dataset size
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42, stratify=y)
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=42, stratify=y_train)

# Apply SMOTE to the reduced dataset
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_sample, y_train_sample)

# XGBoost
xgb_model = XGBClassifier(
    tree_method='gpu_hist',
    gpu_id=0,
    random_state=42
)

xgb_param_dist = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'min_child_weight': [1, 5],
    'gamma': [0.5, 1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

xgb_random_search = RandomizedSearchCV(
    xgb_model, param_distributions=xgb_param_dist, n_iter=10, 
    scoring='roc_auc', n_jobs=1, cv=2, random_state=42, verbose=1
)

xgb_random_search.fit(X_train_resampled, y_train_resampled)
print("Best XGBoost parameters:", xgb_random_search.best_params_)
xgb_best = xgb_random_search.best_estimator_

# LightGBM
lgbm_model = LGBMClassifier(
    device='gpu',
    gpu_platform_id=0,
    gpu_device_id=0,
    random_state=42
)

lgbm_param_dist = {
    'num_leaves': [31, 63],
    'max_depth': [5, 7],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'min_child_samples': [20, 50],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

lgbm_random_search = RandomizedSearchCV(
    lgbm_model, param_distributions=lgbm_param_dist, n_iter=10, 
    scoring='roc_auc', n_jobs=1, cv=2, random_state=42, verbose=1
)

lgbm_random_search.fit(X_train_resampled, y_train_resampled)
print("Best LightGBM parameters:", lgbm_random_search.best_params_)
lgbm_best = lgbm_random_search.best_estimator_

# Evaluate models
def evaluate_model(model, X, y, model_name):
    y_pred = model.predict(X)
    y_pred_proba = model.predict_proba(X)[:, 1]
    print(f"\n{model_name} Results:")
    print(classification_report(y, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y, y_pred_proba)}")

evaluate_model(xgb_best, X_test, y_test, "XGBoost")
evaluate_model(lgbm_best, X_test, y_test, "LightGBM")

# Feature importance
xgb_feature_importance = pd.DataFrame({
    'feature': X_selected.columns,
    'importance': xgb_best.feature_importances_
}).sort_values('importance', ascending=False)

lgbm_feature_importance = pd.DataFrame({
    'feature': X_selected.columns,
    'importance': lgbm_best.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Important Features (XGBoost):")
print(xgb_feature_importance.head(10))

print("\nTop 10 Important Features (LightGBM):")
print(lgbm_feature_importance.head(10))

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best XGBoost parameters: {'subsample': 0.8, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.5, 'colsample_bytree': 1.0}
Fitting 2 folds for each of 10 candidates, totalling 20 fits
[LightGBM] [Info] Number of positive: 26200, number of negative: 26200
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5679
[LightGBM] [Info] Number of data points in the train set: 52400, number of used features: 26
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 19 dense feature groups (1.00 MB) transferred to GPU in 0.001802 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.5000

In [6]:
from imblearn.combine import SMOTETomek
from sklearn.metrics import f1_score, make_scorer

# Use SMOTETomek instead of SMOTE
smote_tomek = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train_sample, y_train_sample)

# Adjust XGBoost parameters
xgb_model = XGBClassifier(
    tree_method='gpu_hist',
    gpu_id=0,
    random_state=42,
    scale_pos_weight=len(y_train_sample[y_train_sample==0]) / len(y_train_sample[y_train_sample==1])
)

xgb_param_dist.update({
    'max_delta_step': [1, 5, 10],
    'scale_pos_weight': [1, 5, 10]
})

# Use F1 score as the scoring metric
f1_scorer = make_scorer(f1_score, average='weighted')

xgb_random_search = RandomizedSearchCV(
    xgb_model, param_distributions=xgb_param_dist, n_iter=20, 
    scoring=f1_scorer, n_jobs=1, cv=3, random_state=42, verbose=1
)

# Similar adjustments for LightGBM...

# After training, find the optimal threshold
from sklearn.metrics import precision_recall_curve

def find_optimal_threshold(y_true, y_pred_proba):
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_proba)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
    return thresholds[np.argmax(f1_scores)]

threshold = find_optimal_threshold(y_test, xgb_best.predict_proba(X_test)[:, 1])
y_pred = (xgb_best.predict_proba(X_test)[:, 1] >= threshold).astype(int)

print("Results with optimal threshold:")
print(classification_report(y_test, y_pred))

Results with optimal threshold:
              precision    recall  f1-score   support

           0       0.93      0.85      0.88     43667
           1       0.38      0.58      0.46      7069

    accuracy                           0.81     50736
   macro avg       0.65      0.72      0.67     50736
weighted avg       0.85      0.81      0.83     50736



In [7]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE

# Assume df is your original dataframe with all features

# Original features (adjust this list based on your original dataset)
original_features = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 
                     'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 
                     'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']

# All features including engineered ones
all_features = X_selected.columns.tolist()  # Assuming X_selected contains all features

# Target variable
y = df['Diabetes_binary']

# Function to train and evaluate model
def train_and_evaluate(X, y, feature_set_name):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    model = XGBClassifier(tree_method='gpu_hist', gpu_id=0, random_state=42)
    model.fit(X_train_resampled, y_train_resampled)
    
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    print(f"\nResults for {feature_set_name}:")
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba)}")
    
    return model

# Train and evaluate model with original features
X_original = df[original_features]
model_original = train_and_evaluate(X_original, y, "Original Features")

# Train and evaluate model with all features (including engineered ones)
X_all = X_selected
model_all = train_and_evaluate(X_all, y, "All Features (including engineered)")

# Compare feature importances
def print_top_features(model, feature_names, top_n=10):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    print(f"\nTop {top_n} features:")
    for f, idx in enumerate(indices[:top_n]):
        print("{0}) {1}: {2:.4f}".format(f + 1, feature_names[idx], importances[idx]))

print_top_features(model_original, original_features)
print_top_features(model_all, all_features)


Results for Original Features:
              precision    recall  f1-score   support

           0       0.93      0.73      0.82     43667
           1       0.29      0.67      0.40      7069

    accuracy                           0.72     50736
   macro avg       0.61      0.70      0.61     50736
weighted avg       0.84      0.72      0.76     50736

ROC AUC Score: 0.7760254733072033

Results for All Features (including engineered):
              precision    recall  f1-score   support

           0       0.88      0.97      0.92     43667
           1       0.52      0.19      0.28      7069

    accuracy                           0.86     50736
   macro avg       0.70      0.58      0.60     50736
weighted avg       0.83      0.86      0.83     50736

ROC AUC Score: 0.8209211149947659

Top 10 features:
1) HvyAlcoholConsump: 0.1673
2) GenHlth: 0.1497
3) HighBP: 0.1447
4) PhysActivity: 0.0714
5) CholCheck: 0.0571
6) NoDocbcCost: 0.0563
7) Fruits: 0.0450
8) Age: 0.0432
9) Smoker: 